# A simple agent-based model

## An interactive ABM

### Use !pip install to install the various libraries used in the program. Press Shift+Enter while on the selected cell of code you want to run.

In [ ]:
!pip install --upgrade pip
!pip install tornado==4.3 --upgrade
!pip install git+https://github.com/AB-CE/abce.git@icis --upgrade 
!pip install plotly

After you have run the installation code, you can now run the agent-based model:

In [ ]:
import abce
import pandas as pd

from firm import Firm
from people import People


params = dict(
    population=1000,
    people_money=1000,
    num_firms=20,
    firm_money=2000,

    num_days=2000,

    l=0.5,  # constant from CS equation

    num_days_buffer=10,  # number of days worth of wages a firm will keep after giving profits

    phi_upper=10,  # phi_upper * demand gives upper bound to inventory
    phi_lower=2,
    excess=1.1,  # if number of workers offered to work for firm exceeds 110% of ideal number, decrease wage
    wage_increment=0.01,
    price_increment=0.01,
    worker_increment=0.01,
    productivity=1,
    wage_acceptance=1)
simulation = abce.Simulation(name='economy', processes=1)
group_of_firms = simulation.build_agents(Firm, "firm", number=params["num_firms"], **params)
people = simulation.build_agents(People, "people", number=1, **params)

for r in range(params["num_days"]):
    simulation.time = r

    group_of_firms.panel_log(variables=['wage', 'ideal_num_workers'], goods=['workers'])
    people.create_labor()

    vacancies_list = list(group_of_firms.publish_vacencies())

    people.send_workers(vacancies_list)

    group_of_firms.production()
    group_of_firms.pay_workers()
    group_of_firms.pay_dividents()
    group_of_firms.send_prices()
    people.get_prices()
    demand = people.buy_goods()

    group_of_firms.sell_goods()
    group_of_firms.determine_bounds(demand=list(demand)[0])
    (group_of_firms + people).print_possessions()
    group_of_firms.determine_wage()
    group_of_firms.expand_or_change_price()
    (people + group_of_firms).destroy_unused_labor()
    people.consumption()
    group_of_firms.determine_profits()

print('done')
path = simulation.path

simulation.finalize()

df = pd.read_csv(path + "/panel_firm.csv")

Once this cell is completed (it might take some time) there will now be a DataFrame with the relevant ABM data in it. By running the next cell you can see the general layout of this dataset.

In [ ]:
df.head()

This next cell of code contains a new function that will let you produce a graph of one of the variables from the DataFrame.

In [ ]:
# Plotting method
import plotly.plotly as py
import plotly.graph_objs as go


def GraphFn(graphing_variable):
    """
    function that takes in graphing variable as parameter and the produces a graph
    using plotly
    """
    x_data = [[] for _ in range(params["num_firms"])]
    y_data = [[] for _ in range(params["num_firms"])]

    for i in range(len(df)):
        name = df["name"][i]
        number = int(name[4:])
        x_data[number].append(df["round"][i])
        y_data[number].append(df[graphing_variable][i])

    data = []    

    for i in range(params["num_firms"]):
        data.append(go.Scatter(x = x_data[i],
                            y = y_data[i],
                            mode = "lines"))
                            #name = ("firm" + str(i))))


    import plotly.offline as offline

    offline.init_notebook_mode()
    offline.iplot(data)

You can now look at the values for a variable from the firms, for example if you run the next cell you will be able to see the time evolution of each firms production:

In [ ]:
GraphFn("production")
    